# Deep Research Project

Agentic AI Project to replicate OpenAI's Deep Research Agentic AI model. Using this reference primarily to start: https://www.analyticsvidhya.com/blog/2025/02/build-your-own-deep-research-agent/

## Project Setup / APIs etc.

Adding Project Root to path to import src files and cofig etc.

In [3]:
import sys
import os

# Get the absolute path to the project root
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))

# Add the project root to the Python path
if project_root not in sys.path:
    sys.path.append(project_root)
    
# Now try importing
from src import get_openai_key

OpenAI API Key

In [4]:
OPENAI_KEY = get_openai_key()

Tavily API Key - Web Search Tool